In [1]:
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.set_num_threads(8)

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "FMNIST",
            'random_class_idx' : False,
            'method': "vanilla",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 50,
            # 'per_task_examples': np.inf,
            'per_task_examples': 200000,
            'per_task_memory_examples': 64,
            'batch_size_train': 256,
            'batch_size_memory': 256,
            # 'batch_size_memory': 256,
            'batch_size_validation': 256,
            'tau': 2.0,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.01,
            'learning_rate_decay_epoch': [30, 50, 70, 90],
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:5' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.0,
            'lambda': 0.0,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=CIFAR10/seed=0_epoch=50_lr=0.01_alpha=0.0_tau=2.0


In [3]:
from datasets import MNIST, FashionMNIST, BiasedMNIST, CIFAR10, CIFAR100
if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                joint = (params['method'] == "joint"),
                                random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 28, 28)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)


[0 1 2 3 4 5 6 7 8 9]
Files already downloaded and verified
Files already downloaded and verified


In [5]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2
from backbones import MLP2Layers2

from backbones.resnet import ResNet18


# backbone = ResNet18(
#     input_dim=input_dim, 
#     output_dim=num_classes,
#     class_idx=class_idx,
#     config=params
#     ).to(params['device'])

backbone = ResNet18(
    input_dim=input_dim, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])

algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

modify resnet for cifar


In [6]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.8495, 'loss': 0.0013676766008138656, 'std': 0.0615, 'EER': -1}
[2] Eval metrics for task 1 >> {'accuracy': 0.895, 'loss': 0.000996992401778698, 'std': 0.02400000000000002, 'EER': -1}
[3] Eval metrics for task 1 >> {'accuracy': 0.913, 'loss': 0.0008939114734530449, 'std': 0.02300000000000002, 'EER': -1}
[4] Eval metrics for task 1 >> {'accuracy': 0.9305000000000001, 'loss': 0.0007453939691185952, 'std': 0.0005000000000000004, 'EER': -1}
[5] Eval metrics for task 1 >> {'accuracy': 0.9325000000000001, 'loss': 0.0007099601924419404, 'std': 0.0015000000000000013, 'EER': -1}
[6] Eval metrics for task 1 >> {'accuracy': 0.9405, 'loss': 0.0006763599701225758, 'std': 0.01849999999999996, 'EER': -1}
[7] Eval metrics for task 1 >> {'accuracy': 0.8855, 'loss': 0.001130737043917179, 'std': 0.09549999999999997, 'EER': -1}
[8] Eval metrics for task 1 >> {'accuracy': 0.9430000000000001, 'loss': 0.0

In [ ]:
metric_manager_callback.meters['EER'].get_eer()

[0.0, 0.11625, 0.1426666666666667, 0.182375, 0.17112]

In [ ]:
metric_manager_callback.meters['std'].get_std()

[0.03049999999999997,
 0.30450646544860094,
 0.16718452878979762,
 0.21805614959225525,
 0.2343910407844122]

In [ ]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.944, 0.   , 0.   , 0.   , 0.   ],
       [0.158, 0.39 , 0.   , 0.   , 0.   ],
       [0.126, 0.495, 0.222, 0.   , 0.   ],
       [0.097, 0.117, 0.31 , 0.634, 0.   ],
       [0.084, 0.068, 0.14 , 0.225, 0.664]])

In [ ]:
metric_manager_callback.meters['accuracy'].compute_overall()

[0.9435, 0.27375, 0.281, 0.289625, 0.23620000000000002]

In [ ]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:0.40481500000000004
EER:0.12248233333333333
std:0.1909276369230132
